In [ ]:
import numpy as np

from scipy.interpolate import RegularGridInterpolator
from scipy.spatial import Delaunay
from scipy.interpolate import griddata

show_map = True
if show_map:
    from matplotlib import pyplot as plt

def extract_timestep_per_second(folder_name):
    f = open(folder_name+"log.lammps", "r")
    for x in f:
        if x[:12] == "Performance:":
            Performance = x
    timestep_per_second = Performance.split()[5]
    return np.float32(timestep_per_second)

possible_combinations = np.loadtxt("lammps_input/possible_combination.dat")

actual_results = []
for nt, nx, ny in np.int32(possible_combinations):
    folder_name = 'nt'+str(nt)+'nx'+str(nx)+'ny'+str(ny)+'/'
    timestep_per_second = extract_timestep_per_second(folder_name)
    actual_results.append([nx, ny, timestep_per_second])
    if nx != ny:
        actual_results.append([ny, nx, timestep_per_second])
actual_results = np.array(actual_results)

# 2D extrapolation
Xi, Yi, Zi = actual_results.T

Pi = np.array([Xi, Yi]).transpose()
tri = Delaunay(Pi)

if show_map:
    fig1, ax = plt.subplots(figsize=(8, 8))
    ax.set_box_aspect(1)
    ax.plot(Xi, Yi, "or", label = "Data")
    ax.triplot(Xi, Yi , tri.simplices.copy())
    plt.show()

N = 100
x = np.linspace(1., 32., N)
y = np.linspace(1., 32., N)
X, Y = np.meshgrid(x, y)
P = np.array([X.flatten(), Y.flatten() ]).transpose()

Z_nearest = griddata(Pi, Zi, P, method = "linear").reshape([N, N])

if show_map:
    fig2, ax = plt.subplots(figsize=(8, 8))
    ax.set_box_aspect(1)
    ax.plot(Xi, Yi, "or", label = "Data")
    ax.triplot(Xi, Yi , tri.simplices.copy())
    ax.contourf(X, Y, Z_nearest, 50)
    plt.show()

performance_extrapolated = []
for nx in np.arange(1, 32+1):
    for ny in np.arange(1, 32+1):
        folder_name = 'nt'+str(nt)+'nx'+str(nx)+'ny'+str(ny)+'/'
        timestep_per_second = Z_nearest[nx][ny]
        print(nx, ny, timestep_per_second)
        performance_extrapolated.append([nx, ny, timestep_per_second])
performance_extrapolated = np.array(performance_extrapolated)
np.savetxt("performance_extrapolated.dat", performance_extrapolated)